# Trabalho 2

## Start snmpd daemon (agent)

In [ ]:
%%bash

sudo service snmpd restart
sudo service snmpd status

# Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## List all mibs

In [ ]:
%%bash

ls -R /usr/share/snmp/mibs

## Show the description of a specific object

In [ ]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/SNMPv2-MIB.txt"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

## Using snmpget

In [ ]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

## Using snmpset

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Comparing snmpwalk vs snmpbulkwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [ ]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## Agent code

In [ ]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import datetime
import socket
import random
import os

# Simulação de informações do dispositivo de rede

def get_device_name():
    return "MyDevice"

def get_device_location():
    return "Servers Room"

def get_device_uptime():
    # Simulando tempo de uptime em segundos
    uptime = datetime.datetime.now() - datetime.datetime(2023, 1, 1, 0, 0)
    uptime_seconds = int(uptime.total_seconds())
    return uptime_seconds * 100

def get_interface_admin_status():
    # Simulando status administrativo da interface (1 = up, 2 = down)
    return random.choice([1, 2])

def get_interface_oper_status():
    # Simulando status operacional da interface (1 = up, 2 = down, 3 = testing)
    return random.choice([1, 2, 3])

def get_interface_in_traffic():
    # Simulando tráfego de entrada em octetos
    return random.randint(1000, 100000)

def get_interface_out_traffic():
    # Simulando tráfego de saída em octetos
    return random.randint(1000, 100000)

def get_interface_in_errors():
    # Simulando número de pacotes recebidos com erro
    return random.randint(0, 10)

def get_cpu_utilization():
    # Simulando utilização da CPU em percentagem
    return random.randint(0, 100)

def get_memory_utilization():
    # Simulando utilização da memória em percentagem
    return random.randint(0, 100)

def get_number_of_sessions():
    # Simulando número de sessões TCP estabelecidas
    return random.randint(10, 50)

def get_number_of_interfaces():
    filepath = '/tmp/custom_interfaces.txt'
    if not os.path.exists(filepath):
        with open(filepath, 'w') as f:
            f.write("5")
        return "5"
    with open(filepath, 'r') as f:
        return f.read()

def set_number_of_interfaces(new_interfaces):
    if not int(new_interfaces):
        return "ERROR"
    
    filepath = '/tmp/custom_interfaces.txt'
    with open(filepath, 'w') as f:
        f.write(new_interfaces)
    return new_interfaces

def main():
    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) < 3:
        print("Usage: agent.py <request-type> <MIB-oid> [type] [<new-value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.1234.1.1.0":
            print(".1.3.6.1.3.1234.1.1.0")
            print("string")
            print(get_device_name())
        elif oid == ".1.3.6.1.3.1234.1.2.0":
            print(".1.3.6.1.3.1234.1.2.0")
            print("string")
            print(get_device_location())
        elif oid == ".1.3.6.1.3.1234.1.3.0":
            print(".1.3.6.1.3.1234.1.3.0")
            print("string")
            print(get_device_uptime())
        elif oid == ".1.3.6.1.3.1234.1.4.0":
            print(".1.3.6.1.3.1234.1.4.0")
            print("string")
            print(get_interface_admin_status())
        elif oid == ".1.3.6.1.3.1234.1.5.0":
            print(".1.3.6.1.3.1234.1.5.0")
            print("string")
            print(get_interface_oper_status())
        elif oid == ".1.3.6.1.3.1234.1.6.0":
            print(".1.3.6.1.3.1234.1.6.0")
            print("string")
            print(get_interface_in_traffic())
        elif oid == ".1.3.6.1.3.1234.1.7.0":
            print(".1.3.6.1.3.1234.1.7.0")
            print("string")
            print(get_interface_out_traffic())
        elif oid == ".1.3.6.1.3.1234.1.8.0":
            print(".1.3.6.1.3.1234.1.8.0")
            print("string")
            print(get_interface_in_errors())
        elif oid == ".1.3.6.1.3.1234.1.9.0":
            print(".1.3.6.1.3.1234.1.9.0")
            print("string")
            print(get_cpu_utilization())
        elif oid == ".1.3.6.1.3.1234.1.10.0":
            print(".1.3.6.1.3.1234.1.10.0")
            print("string")
            print(get_memory_utilization())
        elif oid == ".1.3.6.1.3.1234.1.11.0":
            print(".1.3.6.1.3.1234.1.11.0")
            print("string")
            print(get_number_of_sessions())
        elif oid == ".1.3.6.1.3.1234.1.12.0":
            print(".1.3.6.1.3.1234.1.12.0")
            print("string")
            print(get_number_of_interfaces())
        else:
            print("NONE")
    elif request_type == "-s":  # SET request
        if oid == ".1.3.6.1.3.1234.1.12.0":
            print(".1.3.6.1.3.1234.1.12.0")
            print("string")
            if len(sys.argv) == 5:
                print(set_number_of_interfaces(sys.argv[4]))
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()

## Testing

In [ ]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.2.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.3.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.4.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.5.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.6.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.7.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.8.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.9.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.10.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.11.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.12.0

In [ ]:
%%bash

snmpset -v2c -c private localhost .1.3.6.1.3.1234.1.12.0 s teste

In [ ]:
%%writefile MYMIB.txt

MYMIB DEFINITIONS ::= BEGIN

IMPORTS
    OBJECT-GROUP FROM SNMPv2-CONF
    OBJECT-TYPE, experimental FROM SNMPv2-SMI
    DisplayString FROM SNMPv2-TC;

myMIB MODULE-IDENTITY
    LAST-UPDATED "202406290000Z"
    ORGANIZATION "PUCRS"
    CONTACT-INFO "admin@localhost"
    DESCRIPTION "My MIB"
::= { experimental 1234 }

myObjects OBJECT IDENTIFIER ::= { myMIB 1 }

myName OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Device Name"
    ::= { myObjects 1 }

myLocation OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Device Location"
    ::= { myObjects 2 }

myUptime OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Device Uptime"
    ::= { myObjects 3 }

myAdminStatus OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Interface Admin Status"
    ::= { myObjects 4 }

myOperStatus OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Interface Oper Status"
    ::= { myObjects 5 }

myInTraffic OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Interface In Traffic"
    ::= { myObjects 6 }

myOutTraffic OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Interface Out Traffic"
    ::= { myObjects 7 }

myInErrors OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Interface In Errors"
    ::= { myObjects 8 }

myCPUUtilization OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "CPU Utilization"
    ::= { myObjects 9 }

myMemoryUtilization OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Memory Utilization"
    ::= { myObjects 10 }

mySessionsNumber OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION "Number of Sessions"
    ::= { myObjects 11 }

END

In [ ]:
%%bash

snmpget -v2c -c public -M +. -m +MYMIB localhost myName.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myLocation.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myUptime.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myAdminStatus.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myOperStatus.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myInTraffic.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myOutTraffic.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myInErrors.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myCPUUtilization.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myMemoryUtilization.0
snmpget -v2c -c public -M +. -m +MYMIB localhost mySessionsNumber.0

In [ ]:
%%bash

snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myName.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myLocation.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myUptime.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myAdminStatus.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myOperStatus.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myInTraffic.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myOutTraffic.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myInErrors.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myCPUUtilization.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::myMemoryUtilization.0
snmpgetnext -v2c -c public -M +. -m +MYMIB localhost MYMIB::mySessionsNumber.0

In [ ]:
%%bash

snmpset -v2c -c private -M +. -m +MYMIB localhost mySessionsNumber.0 s 1